In [1]:
import os
import sys
import json
import codecs
import importlib
import sys
sys.path.append("../")
import poses
import numpy as np
import pandas as pd
from PIL import Image

In [4]:
def log_to_df(path="robot_logs/", grasp_only=False, timestamp=0, obj=""):
    '''
    combine 3 logs:
    1. move (csv)
    2. grasp_log (json)
    3. home (csv)
    '''
    global IMAGE
    # required format
    # observation: 6x joint velocities, gripper velocity, gripper force
    # camera: 640x480x3 RGB image
    # alternative: depth image too?

    # list of dictionaries to tfds
    # https://stackoverflow.com/questions/68567630/converting-a-list-of-dictionaries-to-a-tf-dataset

    # parse out object from path
    obj = path.split('_id-')[0].split('_')[1:]
    obj = ' '.join(obj)

    # parse out timestamp from path
    if timestamp == 0:
        timestamp = path.split('_')[0].split('/')[-1]
    
    if "FAIL" in path:
        fail_reason = path.split('-')[-1]

    # load csv to pd df
    home = pd.read_csv(f"{path}/home.csv")
    move = pd.read_csv(f"{path}/move.csv")
    
    # extract 6d poses, x, y, z, rx, ry, rz from actual_TCP_pose_0 to actual_TCP_pose_5
    home_tcp = home[['actual_TCP_pose_0', 'actual_TCP_pose_1', 'actual_TCP_pose_2', 'actual_TCP_pose_3', 'actual_TCP_pose_4', 'actual_TCP_pose_5']]
    move_tcp = move[['actual_TCP_pose_0', 'actual_TCP_pose_1', 'actual_TCP_pose_2', 'actual_TCP_pose_3', 'actual_TCP_pose_4', 'actual_TCP_pose_5']]
    # calculate delta TCP pose from 'target_TCP_pose_0' to 'target_TCP_pose_5' vs 'actual_TCP_pose_0' to 'actual_TCP_pose_5'
    home_tcp_target = home[['target_TCP_pose_0', 'target_TCP_pose_1', 'target_TCP_pose_2', 'target_TCP_pose_3', 'target_TCP_pose_4', 'target_TCP_pose_5']]
    move_tcp_target = move[['target_TCP_pose_0', 'target_TCP_pose_1', 'target_TCP_pose_2', 'target_TCP_pose_3', 'target_TCP_pose_4', 'target_TCP_pose_5']]
    
    def axis_angle_to_quat(pose):
        # convert 6d pose to matrix
        tmat = poses.pose_vec_to_mtrx(pose)
        # convert rot to roll, pitch, yaw
        quat = poses.rotMatx_2_quat(tmat[:3, :3])
        quat = np.array([quat.scalar, quat.vector[0], quat.vector[1], quat.vector[2]])
        p = np.concatenate((pose[:3], quat))
        return p
    
    # very annoyed. bridge uses rpy/delta rpy
    # autolab uses quat for state, delta rpy for action. freaks!
    def axis_angle_to_rpy(pose):
        # convert 6d pose to matrix
        tmat = poses.pose_vec_to_mtrx(pose)
        # convert rot to roll, pitch, yaw
        rpy = poses.rotation_mtrx_to_rpy(tmat[:3, :3])
        # combine with translation, the first 3 elements of pose
        rpy = np.concatenate((pose[:3], rpy))
        return rpy

    # convert 6d axis-angle pose to 6d rpy pose for home_tcp and move_tcp
    home_tcp = home_tcp.apply(axis_angle_to_rpy, axis=1)
    move_tcp = move_tcp.apply(axis_angle_to_rpy, axis=1)
    home_tcp_target = home_tcp_target.apply(axis_angle_to_rpy, axis=1)
    move_tcp_target = move_tcp_target.apply(axis_angle_to_rpy, axis=1)

    home_tcp_delta = home_tcp_target - home_tcp
    move_tcp_delta = move_tcp_target - move_tcp

    # extract timestamp columns and actual_qd columns from actual_q_0 to actual_q_5
    home = home[['timestamp', 'actual_q_0', 'actual_q_1', 'actual_q_2', 'actual_q_3', 'actual_q_4', 'actual_q_5']]
    move = move[['timestamp', 'actual_q_0', 'actual_q_1', 'actual_q_2', 'actual_q_3', 'actual_q_4', 'actual_q_5']]
    # rename actual_q_0 to actual_q_5 to q0 to q5
    home.rename(columns={'actual_q_0': 'q0', 'actual_q_1': 'q1', 'actual_q_2': 'q2', 'actual_q_3': 'q3', 'actual_q_4': 'q4', 'actual_q_5': 'q5'}, inplace=True)
    move.rename(columns={'actual_q_0': 'q0', 'actual_q_1': 'q1', 'actual_q_2': 'q2', 'actual_q_3': 'q3', 'actual_q_4': 'q4', 'actual_q_5': 'q5'}, inplace=True)
    # home = home[['timestamp']]
    # move = move[['timestamp']]

    
    # assign to home and move dfs
    # home tcp has become a single column df, each cell containing a 6-element list, need to split back into 'actual_TCP_pose_0', 'actual_TCP_pose_1', 'actual_TCP_pose_2', 'actual_TCP_pose_3', 'actual_TCP_pose_4', 'actual_TCP_pose_5'
    home_tcp = pd.DataFrame(home_tcp.tolist(), columns=['x', 'y', 'z', 'rx', 'ry', 'rz'])
    move_tcp = pd.DataFrame(move_tcp.tolist(), columns=['x', 'y', 'z', 'rx', 'ry', 'rz'])
    home_tcp_delta = pd.DataFrame(home_tcp_delta.tolist(), columns=['dx', 'dy', 'dz', 'drx', 'dry', 'drz'])
    move_tcp_delta = pd.DataFrame(move_tcp_delta.tolist(), columns=['dx', 'dy', 'dz', 'drx', 'dry', 'drz'])

    home[['x', 'y', 'z', 'rx', 'ry', 'rz']] = home_tcp
    move[['x', 'y', 'z', 'rx', 'ry', 'rz']] = move_tcp
    home[['dx', 'dy', 'dz', 'drx', 'dry', 'drz']] = home_tcp_delta
    move[['dx', 'dy', 'dz', 'drx', 'dry', 'drz']] = move_tcp_delta

    # load json
    grasp_pth = f"{path}/grasp.json"
    obj_text = codecs.open(grasp_pth, 'r', encoding='utf-8').read()
    gl = np.array(json.loads(obj_text))
    grasp_log = pd.DataFrame(gl.tolist()) # convert to pd df
    # keep only timestamp, aperture, gripper_vel, and contact_force cols, add actual_qd columns from actual_qd_0 to actual_qd_6 from the last row of move to grasp_log

    gl_copy = grasp_log.copy()
    grasp_log = grasp_log[['timestamp', 'aperture', 'applied_force', 'contact_force']]
    # set the ceiling of applied_force to be 32.4N
    grasp_log['applied_force'] = grasp_log['applied_force'].clip(0.0, 32.4)
    # look one row ahead to get the n+1 aperture and applied_force, subtract current aperture and applied force, and set to d_aperture, d_applied_force
    grasp_log['d_aperture'] = grasp_log['aperture'].diff().shift(-1)
    grasp_log['d_applied_force'] = grasp_log['applied_force'].diff().shift(-1)
    # set the last row's d_aperture and d_applied_force to 0
    grasp_log.iloc[-1, -2:] = 0.0
    grasp_log[['q0', 'q1', 'q2', 'q3', 'q4', 'q5', 'x', 'y', 'z', 'rx', 'ry', 'rz']] = move.iloc[-1][['q0', 'q1', 'q2', 'q3', 'q4', 'q5', 'x', 'y', 'z', 'rx', 'ry', 'rz']]
    grasp_log[['dx', 'dy', 'dz', 'drx', 'dry', 'drz']] = 0.0
    # reorder columns so that it is: timestamp, x, y, z, qx, qy, qz, qw, dx, dy, dz, dqx, dqy, dqz, dqw, aperture, d_aperture, applied_force, d_applied_force
    grasp_log = grasp_log[['timestamp', 'q0', 'q1', 'q2', 'q3', 'q4', 'q5', 'x', 'y', 'z', 'rx', 'ry', 'rz', 'dx', 'dy', 'dz', 'drx', 'dry', 'drz', 'aperture', 'd_aperture', 'applied_force', 'd_applied_force', 'contact_force']]
    # grasp_log.fillna(0.0, inplace=True)
    # df = grasp_log
    # print(grasp_log[['applied_force', 'd_applied_force', 'aperture', 'd_aperture']])

    # add columns to move and home dfs
    move['aperture'] = 100.0 # set to 100.0mm
    move['d_aperture'] = 0.0
    move['applied_force'] = 0.0
    move['d_applied_force'] = 0.0
    move['contact_force'] = 0.0
    
    home['aperture'] = grasp_log['aperture'].iloc[-1]
    home['d_aperture'] = 0.0
    home['applied_force'] = grasp_log['applied_force'].iloc[-1]
    home['d_applied_force'] = 0.0
    home['contact_force'] = grasp_log['contact_force'].iloc[-1]

    # make last 'd_applied_force' of move be 0 + first 'applied_force' of grasp_log
    move['d_applied_force'].iloc[-1] = grasp_log['applied_force'].iloc[0]
    # make last 'd_aperture' of move be  first 'aperture' of grasp_log - 100mm
    move['d_aperture'].iloc[-1] = grasp_log['aperture'].iloc[0] - 100

    # add task column to move, grasp_log, and home
    move_task = f"move to grasp {obj}"
    grasp_task = f"grasp {obj}"
    home_task = f"move back to original position with {obj}"
    task = f"grasp {obj} and return to original position"
    if grasp_only:
        task = grasp_task
    move['subtask'] = move_task
    move['task'] = task
    grasp_log['subtask'] = grasp_task
    grasp_log['task'] = task
    home['subtask'] = home_task
    home['task'] = task

    # make last subtask of move be grasp_task
    move['subtask'].iloc[-1] = grasp_task

    def extract_timestamp(filename):
        # extracting the float
        img = filename.split('_')[1].split('.')[:2]
        return float(f"{img[0]}.{img[1]}")

    # # get initial values, iteratively subtract 0.1s from last_move and last_home and assign to to move and home timestamps, from last to first
    move_workspace_img = sorted([extract_timestamp(f) for f in os.listdir(f"{path}/workspace_img") if f.startswith("0_")])
    move_wrist_img = sorted([extract_timestamp(f) for f in os.listdir(f"{path}/wrist_img") if f.startswith("0_")])
    last_move_workspace_img = move_workspace_img[-1]
    last_move_wrist_img = move_wrist_img[-1]
    last_move = max(last_move_workspace_img, last_move_wrist_img)

    home_workspace_img = sorted([extract_timestamp(f) for f in os.listdir(f"{path}/workspace_img") if f.startswith("2_")])
    home_wrist_img = sorted([extract_timestamp(f) for f in os.listdir(f"{path}/wrist_img") if f.startswith("2_")])
    last_home_workspace_img = home_workspace_img[-1]
    last_home_wrist_img = home_wrist_img[-1]
    last_home = max(last_home_workspace_img, last_home_wrist_img)

    for i in np.arange(len(move)-1, -1, -1):
        # iterate backwards through move timestamps
        move.loc[i, 'timestamp'] = last_move
        last_move -= 0.096
    
    for i in np.arange(len(home)-1, -1, -1):
        # iterate backwards through home timestamps
        home.loc[i, 'timestamp'] = last_home
        last_home -= 0.096

    grasp_workspace_img = sorted([extract_timestamp(f) for f in os.listdir(f"{path}/workspace_img") if f.startswith("1_")])
    grasp_wrist_img = sorted([extract_timestamp(f) for f in os.listdir(f"{path}/wrist_img") if f.startswith("1_")])
    
    def timestamp_to_filename(timestamp, index):
        # converting the float to a string
        return f"{index}_{str(timestamp)}.jpeg"

    move_workspace_interpolated = []
    grasp_workspace_interpolated = []
    home_workspace_interpolated = []
    move_wrist_interpolated = []
    grasp_wrist_interpolated = []
    home_wrist_interpolated = []
    # go thropugh each timestamp in move, grasp, and home
    # find the closest img timestamp and append to move_interpolated, grasp_interpolated, and home_interpolated
    for m in move['timestamp']:
        move_workspace_interpolated.append(timestamp_to_filename(min(move_workspace_img, key=lambda x:abs(x-m)), 0))
        move_wrist_interpolated.append(timestamp_to_filename(min(move_wrist_img, key=lambda x:abs(x-m)), 0))
    for g in grasp_log['timestamp']:
        grasp_workspace_interpolated.append(timestamp_to_filename(min(grasp_workspace_img, key=lambda x:abs(x-g)), 1))
        grasp_wrist_interpolated.append(timestamp_to_filename(min(grasp_wrist_img, key=lambda x:abs(x-g)), 1))
    for h in home['timestamp']:
        home_workspace_interpolated.append(timestamp_to_filename(min(home_workspace_img, key=lambda x:abs(x-h)), 2))
        home_wrist_interpolated.append(timestamp_to_filename(min(home_wrist_img, key=lambda x:abs(x-h)), 2))

    # make new columns in dataframes
    move['img'] = move_workspace_interpolated
    move['wrist_img'] = move_wrist_interpolated
    grasp_log['img'] = grasp_workspace_interpolated
    grasp_log['wrist_img'] = grasp_wrist_interpolated
    home['img'] = home_workspace_interpolated
    home['wrist_img'] = home_wrist_interpolated

    # write move, grasp_log, and home to csv files
    move.to_csv(f"{path}/0.csv", index=False)
    grasp_log.to_csv(f"{path}/1.csv", index=False)
    home.to_csv(f"{path}/2.csv", index=False)

    # load actual image, not just the filename, cast from jpeg to numpy array
    move['img'] = [np.array(Image.open(f"{path}/workspace_img/{i}")) for i in move['img']]
    move['wrist_img'] = [np.array(Image.open(f"{path}/wrist_img/{i}")) for i in move['wrist_img']]
    grasp_log['img'] = [np.array(Image.open(f"{path}/workspace_img/{i}")) for i in grasp_log['img']]
    grasp_log['wrist_img'] = [np.array(Image.open(f"{path}/wrist_img/{i}")) for i in grasp_log['wrist_img']]
    home['img'] = [np.array(Image.open(f"{path}/workspace_img/{i}")) for i in home['img']]
    home['wrist_img'] = [np.array(Image.open(f"{path}/wrist_img/{i}")) for i in home['wrist_img']]
    
    # write move, grasp_log, and home to csv files
    # move.to_csv(f"{path}/0.csv", index=False)
    # grasp_log.to_csv(f"{path}/1.csv", index=False)
    # home.to_csv(f"{path}/2.csv", index=False)

    # # combine dataframes vertically
    df = pd.concat([move, grasp_log, home], axis=0)
    # # round all float columns to 6 decimal places
    df = df.round(6)
    # # save df to csv
    # df.to_csv(f"{path}/trajectory.csv", index=False)

    # now we normalize aperture, d_aperture, applied_force, d_applied force, and contact_force
    # aperture: 0 - 100mm
    # force: 0 - 16.2N
    # scale aperture and d_aperture
    df['aperture'] = df['aperture']/100.0 # mm to dm
    df['d_aperture'] = df['d_aperture']/100.0 # mm to dm
    # scale force and d_force
    df['applied_force'] = df['applied_force']/100.0 #N to daN
    df['contact_force'] = df['contact_force']/100.0 #N to daN
    df['d_applied_force'] = df['d_applied_force']/100.0 #N to daN

    # print if any of d_aperture or d_applied_force > 1.0 or < -1.0
    print(df[(df['d_aperture'] > 1.0) | (df['d_aperture'] < -1.0)])
    print(df[(df['d_applied_force'] > 1.0) | (df['d_applied_force'] < -1.0)])
    columns = ['timestamp', 'q0', 'q1', 'q2', 'q3', 'q4', 'q5', 'x', 'y', 'z', 'rx', 'ry', 'rz', 'dx', 'dy', 'dz', 'drx', 'dry', 'drz', 'aperture', 'd_aperture', 'applied_force', 'd_applied_force', 'contact_force', 'subtask', 'task', 'img', 'wrist_img']
    # make sure columns are in the right order
    df = df[columns]
    # df['applied_force']   = (df['applied_force']/ 16.2).clip(0.0, 1.0)
    # df['d_applied_force'] = (df['d_applied_force']/ 16.2).clip(0.0, 1.0)
    # df['contact_force'] = (df['contact_force']/16.2).clip(0.0, 1.0)
    # print(df[['applied_force', 'd_applied_force', 'aperture', 'd_aperture']])

    # get rows where subtask is grasp_task
    grasp_df = df[df['subtask'] == grasp_task]
    if grasp_only: df = grasp_df

    # return df as pickled np array
    df_pkl = df.to_numpy()
    # np.save(f"{path}/episode_{obj}_{timestamp}.npy", df_pkl)
    np.save(f"data/train/episode_{obj}_{timestamp}.npy", df_pkl)
    # np.save(f"data/fails/episode_{obj}_{timestamp}_FAIL-{fail_reason}.npy", df_pkl)

    return move, grasp_log, home

In [3]:
def grasp_debug(path="robot_logs/", timestamp=0, obj=""):
    grasp_pth = f"{path}/grasp.json"
    obj_text = codecs.open(grasp_pth, 'r', encoding='utf-8').read()
    gl = np.array(json.loads(obj_text))
    grasp_log = pd.DataFrame(gl.tolist()) # convert to pd df

    grasp_log = grasp_log[['timestamp', 'aperture', 'applied_force', 'contact_force']]
    # look one row ahead to get the n+1 aperture and applied_force, subtract current aperture and applied force, and set to d_aperture, d_applied_force
    grasp_log['d_aperture'] = grasp_log['aperture'].diff().shift(-1)
    grasp_log['d_applied_force'] = grasp_log['applied_force'].diff().shift(-1)
    grasp_log.iloc[-1, -2:] = 0.0
    # set NaN to 0
    # grasp_log.fillna(0.0, inplace=True)
    df = grasp_log
    print(df[['applied_force', 'd_applied_force', 'aperture', 'd_aperture']])



In [38]:
# log_to_df(path=f"trajectories/1727459089.0012171_bottle_cap_id-0")

In [5]:
# fdf = log_to_df(timestamp=timestamp, obj="FIX_test", path="fix_test")
path = "1726689405.2236292_red_button_id-0"
# path = "1726689855.8823   204_yellow_block_id-0"
# iterate through all trajectory folders in path
path = "/home/wxie/workspace/deligrasp_dataset_builder/deligrasp_dataset/trajectories"
path = "trajectories"
i = 0
for folder in os.listdir(path):
    i +=1
    # if i > 3:
    #     break
    # skip if .npy file already exists in data/train
    obj = path.split('_id-')[0].split('_')[1:]
    obj = ' '.join(obj)
    timestamp = path.split('_')[0].split('/')[-1]
    if os.path.exists(f"data/train/episode_{obj}_{timestamp}.npy"):
        continue
    if os.path.isdir(f"{path}/{folder}"):
        print(f"{path}/{folder}")
        # move, grasp_log, home = log_to_df(path=f"{path}/{folder}", grasp_only=True)
        move, grasp_log, home = log_to_df(path=f"{path}/{folder}")
        # grasp_debug(path=f"{path}/{folder}")
        # print(move, grasp_log, home)
        # break
# m,g,h = log_to_df(path=path)

trajectories/1726689405.2236292_red_button_id-0


C:\Users\willi\AppData\Local\Temp\ipykernel_10196\592521529.py:147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  move['subtask'].iloc[-1] = grasp_task


Empty DataFrame
Columns: [timestamp, q0, q1, q2, q3, q4, q5, x, y, z, rx, ry, rz, dx, dy, dz, drx, dry, drz, aperture, d_aperture, applied_force, d_applied_force, contact_force, subtask, task, img, wrist_img]
Index: []

[0 rows x 28 columns]
Empty DataFrame
Columns: [timestamp, q0, q1, q2, q3, q4, q5, x, y, z, rx, ry, rz, dx, dy, dz, drx, dry, drz, aperture, d_aperture, applied_force, d_applied_force, contact_force, subtask, task, img, wrist_img]
Index: []

[0 rows x 28 columns]
trajectories/1726689855.8823204_yellow_block_id-0


C:\Users\willi\AppData\Local\Temp\ipykernel_10196\592521529.py:147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  move['subtask'].iloc[-1] = grasp_task


Empty DataFrame
Columns: [timestamp, q0, q1, q2, q3, q4, q5, x, y, z, rx, ry, rz, dx, dy, dz, drx, dry, drz, aperture, d_aperture, applied_force, d_applied_force, contact_force, subtask, task, img, wrist_img]
Index: []

[0 rows x 28 columns]
Empty DataFrame
Columns: [timestamp, q0, q1, q2, q3, q4, q5, x, y, z, rx, ry, rz, dx, dy, dz, drx, dry, drz, aperture, d_aperture, applied_force, d_applied_force, contact_force, subtask, task, img, wrist_img]
Index: []

[0 rows x 28 columns]
trajectories/1726691753.9317017_cardboard_box_id-0


...

Empty DataFrame
Columns: [timestamp, q0, q1, q2, q3, q4, q5, x, y, z, rx, ry, rz, dx, dy, dz, drx, dry, drz, aperture, d_aperture, applied_force, d_applied_force, contact_force, subtask, task, img, wrist_img]
Index: []

[0 rows x 28 columns]
Empty DataFrame
Columns: [timestamp, q0, q1, q2, q3, q4, q5, x, y, z, rx, ry, rz, dx, dy, dz, drx, dry, drz, aperture, d_aperture, applied_force, d_applied_force, contact_force, subtask, task, img, wrist_img]
Index: []

[0 rows x 28 columns]
trajectories/1727459089.0012171_bottle_cap_id-0
Empty DataFrame
Columns: [timestamp, q0, q1, q2, q3, q4, q5, x, y, z, rx, ry, rz, dx, dy, dz, drx, dry, drz, aperture, d_aperture, applied_force, d_applied_force, contact_force, subtask, task, img, wrist_img]
Index: []

[0 rows x 28 columns]
Empty DataFrame
Columns: [timestamp, q0, q1, q2, q3, q4, q5, x, y, z, rx, ry, rz, dx, dy, dz, drx, dry, drz, aperture, d_aperture, applied_force, d_applied_force, contact_force, subtask, task, img, wrist_img]
Index: []

[0 row

C:\Users\willi\AppData\Local\Temp\ipykernel_10196\592521529.py:147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  move['subtask'].iloc[-1] = grasp_task
C:\Users\willi\AppData\Local\Temp\ipykernel_10196\592521529.py:147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  move['subtask'].iloc[-1] = grasp_task


Empty DataFrame
Columns: [timestamp, q0, q1, q2, q3, q4, q5, x, y, z, rx, ry, rz, dx, dy, dz, drx, dry, drz, aperture, d_aperture, applied_force, d_applied_force, contact_force, subtask, task, img, wrist_img]
Index: []

[0 rows x 28 columns]
Empty DataFrame
Columns: [timestamp, q0, q1, q2, q3, q4, q5, x, y, z, rx, ry, rz, dx, dy, dz, drx, dry, drz, aperture, d_aperture, applied_force, d_applied_force, contact_force, subtask, task, img, wrist_img]
Index: []

[0 rows x 28 columns]
trajectories/1727459334.0820029_plastic_bottle_id-0
Empty DataFrame
Columns: [timestamp, q0, q1, q2, q3, q4, q5, x, y, z, rx, ry, rz, dx, dy, dz, drx, dry, drz, aperture, d_aperture, applied_force, d_applied_force, contact_force, subtask, task, img, wrist_img]
Index: []

[0 rows x 28 columns]
Empty DataFrame
Columns: [timestamp, q0, q1, q2, q3, q4, q5, x, y, z, rx, ry, rz, dx, dy, dz, drx, dry, drz, aperture, d_aperture, applied_force, d_applied_force, contact_force, subtask, task, img, wrist_img]
Index: []

[0

C:\Users\willi\AppData\Local\Temp\ipykernel_10196\592521529.py:147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  move['subtask'].iloc[-1] = grasp_task
C:\Users\willi\AppData\Local\Temp\ipykernel_10196\592521529.py:147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  move['subtask'].iloc[-1] = grasp_task


Empty DataFrame
Columns: [timestamp, q0, q1, q2, q3, q4, q5, x, y, z, rx, ry, rz, dx, dy, dz, drx, dry, drz, aperture, d_aperture, applied_force, d_applied_force, contact_force, subtask, task, img, wrist_img]
Index: []

[0 rows x 28 columns]
Empty DataFrame
Columns: [timestamp, q0, q1, q2, q3, q4, q5, x, y, z, rx, ry, rz, dx, dy, dz, drx, dry, drz, aperture, d_aperture, applied_force, d_applied_force, contact_force, subtask, task, img, wrist_img]
Index: []

[0 rows x 28 columns]


In [17]:
import importlib
importlib.reload(deligrasp_dataset_dataset_builder)

<module 'deligrasp_dataset_dataset_builder' from 'c:\\workspace\\deligrasp_dataset_builder\\deligrasp_dataset\\deligrasp_dataset_dataset_builder.py'>

In [6]:
# build datasets
import tensorflow_datasets as tfds

# Import your custom dataset builder class
import deligrasp_dataset_dataset_builder

# Instantiate and build the dataset
builder = tfds.builder('DeligraspDataset')  # 'my_dataset' should match the class name in the Python file
builder.download_and_prepare()  # Builds the dataset, as tfds build would do via CLI

# Load the dataset
ds = builder.as_dataset()

# You can now inspect and use the dataset
print(ds)


Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\willi\tensorflow_datasets\deligrasp_dataset\incomplete.7KBVXI_1.0.0\deligrasp_dataset-train…

Dataset deligrasp_dataset downloaded and prepared to C:\Users\willi\tensorflow_datasets\deligrasp_dataset\1.0.0. Subsequent calls will reuse this data.
{'train': <_PrefetchDataset element_spec={'episode_metadata': {'file_path': TensorSpec(shape=(), dtype=tf.string, name=None)}, 'steps': DatasetSpec({'action': TensorSpec(shape=(9,), dtype=tf.float64, name=None), 'action_dict': {'cartesian_position': TensorSpec(shape=(6,), dtype=tf.float64, name=None), 'gripper_force': TensorSpec(shape=(1,), dtype=tf.float64, name=None), 'gripper_position': TensorSpec(shape=(1,), dtype=tf.float64, name=None), 'rotation': TensorSpec(shape=(3,), dtype=tf.float64, name=None), 'translation': TensorSpec(shape=(3,), dtype=tf.float64, name=None)}, 'discount': TensorSpec(shape=(), dtype=tf.float32, name=None), 'is_first': TensorSpec(shape=(), dtype=tf.bool, name=None), 'is_last': TensorSpec(shape=(), dtype=tf.bool, name=None), 'is_terminal': TensorSpec(shape=(), dtype=tf.bool, name=None), 'language_embedding': T

In [82]:
def extract_timestamp(filename):
    # extracting the float
    img = filename.split('_')[1].split('.')[:2]
    return float(f"{img[0]}.{img[1]}")

def timestamp_to_filename(timestamp, index):
    # converting the float to a string
    return f"{index}_{str(timestamp)}.jpeg"

def get_last_timestamps(path=""):
    # we're gonna retrieve timestamps from the image file names in the workspace_img and wrist_img folders
    # image filenames are structured as: <0,1,2>_<epoch_timestamp>.jpeg, 0,1,2 correspond to move, grasp, home respectively
    # we're gonna use the last epoch timestamp from the move and home images to assign timestamps to the move and home dataframes
    # using whichever is later, between the workspace and wrist images
    
    # for the move images, we're going to grab all the images from the path that begin with 0_
    # for the home images, we're going to grab all the images from the path that begin with 2_
    # we're going to sort the images in ascending order
    # we're going to grab the last image in the list
    # we're going to extract the epoch timestamp from the image filename
    move_workspace_img = sorted([extract_timestamp(f) for f in os.listdir(f"{path}/workspace_img") if f.startswith("0_")])
    move_wrist_img = sorted([extract_timestamp(f) for f in os.listdir(f"{path}/wrist_img") if f.startswith("0_")])
    last_move_workspace_img = move_workspace_img[-1]
    last_move_wrist_img = move_wrist_img[-1]
    last_move = max(last_move_workspace_img, last_move_wrist_img)

    home_workspace_img = sorted([extract_timestamp(f) for f in os.listdir(f"{path}/workspace_img") if f.startswith("2_")])
    home_wrist_img = sorted([extract_timestamp(f) for f in os.listdir(f"{path}/wrist_img") if f.startswith("2_")])
    last_home_workspace_img = home_workspace_img[-1]
    last_home_wrist_img = home_wrist_img[-1]
    last_home = max(last_home_workspace_img, last_home_wrist_img)

    return last_move, last_home

def interpolate_timestamps(path):
    # we are going to find the img timestamp that is closest to each row's timestamp
    move_timestamps = pd.read_csv(f"{path}/0.csv")['timestamp']
    grasp_timestamps = pd.read_csv(f"{path}/1.csv")['timestamp']
    home_timestamps = pd.read_csv(f"{path}/2.csv")['timestamp']
    move_workspace_img = sorted([extract_timestamp(f) for f in os.listdir(f"{path}/workspace_img") if f.startswith("0_")])
    move_wrist_img = sorted([extract_timestamp(f) for f in os.listdir(f"{path}/wrist_img") if f.startswith("0_")])
    home_workspace_img = sorted([extract_timestamp(f) for f in os.listdir(f"{path}/workspace_img") if f.startswith("2_")])
    home_wrist_img = sorted([extract_timestamp(f) for f in os.listdir(f"{path}/wrist_img") if f.startswith("2_")])
    grasp_workspace_img = sorted([extract_timestamp(f) for f in os.listdir(f"{path}/workspace_img") if f.startswith("1_")])
    grasp_wrist_img = sorted([extract_timestamp(f) for f in os.listdir(f"{path}/wrist_img") if f.startswith("1_")])
    move_workspace_interpolated = []
    grasp_workspace_interpolated = []
    home_workspace_interpolated = []
    move_wrist_interpolated = []
    grasp_wrist_interpolated = []
    home_wrist_interpolated = []
    # go thropugh each timestamp in move, grasp, and home
    # find the closest img timestamp and append to move_interpolated, grasp_interpolated, and home_interpolated
    for m in move_timestamps:
        move_workspace_interpolated.append(timestamp_to_filename(min(move_workspace_img, key=lambda x:abs(x-m)), 0))
        move_wrist_interpolated.append(timestamp_to_filename(min(move_wrist_img, key=lambda x:abs(x-m)), 0))
    for g in grasp_timestamps:
        grasp_workspace_interpolated.append(timestamp_to_filename(min(grasp_workspace_img, key=lambda x:abs(x-g)), 1))
        grasp_wrist_interpolated.append(timestamp_to_filename(min(grasp_wrist_img, key=lambda x:abs(x-g)), 1))
    for h in home_timestamps:
        home_workspace_interpolated.append(timestamp_to_filename(min(home_workspace_img, key=lambda x:abs(x-h)), 2))
        home_wrist_interpolated.append(timestamp_to_filename(min(home_wrist_img, key=lambda x:abs(x-h)), 2))

    # make new columns in dataframes
    move['img'] = move_workspace_interpolated
    move['wrist_img'] = move_wrist_interpolated
    grasp['img'] = grasp_workspace_interpolated
    grasp['wrist_img'] = grasp_wrist_interpolated
    home['img'] = home_workspace_interpolated
    home['wrist_img'] = home_wrist_interpolated

    return move_workspace_interpolated, move_wrist_interpolated, grasp_workspace_interpolated, grasp_wrist_interpolated, home_workspace_interpolated, home_wrist_interpolated

# get_last_timestamps(path)
interpolate_timestamps(path)

    

NameError: name 'move' is not defined

In [69]:
def as_gif(images, path='temp.gif'):
  # Render the images as the gif:
  images[0].save(path, save_all=True, append_images=images[1:], duration=1000, loop=0)
  gif_bytes = open(path,'rb').read()
  return gif_bytes

# images = [step['observation']['image'] for step in episode['steps']]
# images = [Image.fromarray(image.numpy()) for image in images]
# display.Image(as_gif(images))
for step in episode['steps']:
    print(step['timestamp'])
    print(step['action'])
    print(step['observation'])
    print(step['language_instruction'])
    print(step['reward'])
    print(step['is_terminal'])
    print(step['is_first'])
    print(step['is_last'])
    print("\n")

TypeError: string indices must be integers, not 'str'